In [1]:
from common_methods import *
import torch.nn.functional as F
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from itertools import product
import json
import math
from datasets import load_dataset

[nltk_data] Downloading package punkt to
[nltk_data]     /home/nikhilanand/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/nikhilanand/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nikhilanand/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
ds_dict = load_dataset("pminervini/NQ-Swap")

In [3]:
ds = ds_dict['dev']

In [4]:
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,torch_dtype=torch.float16).eval()
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
device = torch.device("cuda")
model.to(device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [5]:
ds[0]

{'question': 'how many episodes are in chicago fire season 4',
 'org_context': '<P> The fourth season of Chicago Fire , an American drama television series with executive producer Dick Wolf , and producers Derek Haas , Michael Brandt , and Matt Olmstead , was ordered on February 5 , 2015 , by NBC , and premiered on October 13 , 2015 and concluded on May 17 , 2016 . The season contained 23 episodes . </P>',
 'org_answer': ['23'],
 'sub_context': '<P> The fourth season of Chicago Fire , an American drama television series with executive producer Dick Wolf , and producers Derek Haas , Michael Brandt , and Matt Olmstead , was ordered on February 5 , 2015 , by NBC , and premiered on October 13 , 2015 and concluded on May 17 , 2016 . The season contained 775 episodes . </P>',
 'sub_answer': ['775']}

In [6]:
prompt = "[INST]"+ds[0]['sub_context']+ds[0]['question']+"[/INST]"
prompt

'[INST]<P> The fourth season of Chicago Fire , an American drama television series with executive producer Dick Wolf , and producers Derek Haas , Michael Brandt , and Matt Olmstead , was ordered on February 5 , 2015 , by NBC , and premiered on October 13 , 2015 and concluded on May 17 , 2016 . The season contained 775 episodes . </P>how many episodes are in chicago fire season 4[/INST]'

In [7]:
regular_decoding(model,tokenizer,prompt,debug=True,max_tokens=100,show_tqdm=True, return_prob=False)

prompt:  [INST]<P> The fourth season of Chicago Fire , an American drama television series with executive producer Dick Wolf , and producers Derek Haas , Michael Brandt , and Matt Olmstead , was ordered on February 5 , 2015 , by NBC , and premiered on October 13 , 2015 and concluded on May 17 , 2016 . The season contained 775 episodes . </P>how many episodes are in chicago fire season 4[/INST]


  0%|          | 0/100 [00:00<?, ?it/s]We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/sensei-fs/users/nikhilanand/6auto_evaluation/common_methods.py:121: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  last_token_probs = F.softmax(last_token_logits)
 66%|██████▌   | 66/100 [00:04<00:02, 16.10it/s]

output:  Chicago Fire Season 4 contains a total of 22 episodes. The information you provided about "775 episodes" seems to be incorrect. Each season of Chicago Fire typically has around 23-24 episodes, so it's likely that there was some confusion about the total number of episodes across all seasons.


In [8]:
# LOOP
outputs = []

for i in range(30):
    dic = {'id':i}
    dic.update(ds[i])
    prompt = "[INST]"+dic['sub_context']+dic['question']+"[/INST]"
    out = regular_decoding(model,tokenizer,prompt,debug=False,max_tokens=100,show_tqdm=True, return_prob=False)
    dic['outputs'] = {'regular_output': out}
    outputs.append(dic)

with open("runs/outputs.json",'w') as file:
    json.dump(outputs,file)

 42%|████▏     | 42/100 [00:01<00:02, 23.49it/s]


KeyboardInterrupt: 

In [6]:
failure_ids = [0,1,3,5,6,7,12,17,25,27,28]

In [11]:
# LOOP
outputs = []

for i in failure_ids:
    dic = {'id':i}
    dic.update(ds[i])
    prompt = "[INST]"+dic['sub_context']+dic['question']+"[/INST]"
    out = regular_decoding(model,tokenizer,prompt,debug=False,max_tokens=100,show_tqdm=True, return_prob=False)
    dic['outputs'] = {'regular_output': out}
    outputs.append(dic)

with open("eval_data/failure_outputs.json",'w') as file:
    json.dump(outputs,file)

100%|██████████| 100/100 [00:04<00:00, 22.95it/s]


In [7]:
# LOOP
outputs = []

for i in range(30):
    if i in failure_ids:
        continue
    dic = {'id':i}
    dic.update(ds[i])
    prompt = "[INST]"+dic['sub_context']+dic['question']+"[/INST]"
    out = regular_decoding(model,tokenizer,prompt,debug=False,max_tokens=100,show_tqdm=True, return_prob=False)
    dic['outputs'] = {'regular_output': out}
    outputs.append(dic)

with open("eval_data/success_outputs.json",'w') as file:
    json.dump(outputs,file)

  0%|          | 0/100 [00:00<?, ?it/s]We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/sensei-fs/users/nikhilanand/6auto_evaluation/common_methods.py:121: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  last_token_probs = F.softmax(last_token_logits)
 98%|█████████▊| 98/100 [00:04<00:00, 23.78it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'runs/success_outputs.json'

In [8]:
with open("eval_data/success_outputs.json",'w') as file:
    json.dump(outputs,file)

In [27]:
from datasets import load_dataset,Dataset
amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2")

Repo card metadata block was not found. Setting CardData to empty.


In [28]:
amnesty_qa['eval']

Dataset({
    features: ['question', 'ground_truth', 'answer', 'contexts'],
    num_rows: 20
})

In [29]:
subset = Dataset.from_dict(amnesty_qa['eval'][:15])
print(len(subset))

15


In [30]:
subset[0]

{'question': 'What are the global implications of the USA Supreme Court ruling on abortion?',
 'ground_truth': "The global implications of the USA Supreme Court ruling on abortion are significant. The ruling has led to limited or no access to abortion for one in three women and girls of reproductive age in states where abortion access is restricted. These states also have weaker maternal health support, higher maternal death rates, and higher child poverty rates. Additionally, the ruling has had an impact beyond national borders due to the USA's geopolitical and cultural influence globally. Organizations and activists worldwide are concerned that the ruling may inspire anti-abortion legislative and policy attacks in other countries. The ruling has also hindered progressive law reform and the implementation of abortion guidelines in certain African countries. Furthermore, the ruling has created a chilling effect in international policy spaces, empowering anti-abortion actors to undermin

In [36]:
amnesty_qa['eval'][:1]

{'question': ['What are the global implications of the USA Supreme Court ruling on abortion?'],
 'ground_truth': ["The global implications of the USA Supreme Court ruling on abortion are significant. The ruling has led to limited or no access to abortion for one in three women and girls of reproductive age in states where abortion access is restricted. These states also have weaker maternal health support, higher maternal death rates, and higher child poverty rates. Additionally, the ruling has had an impact beyond national borders due to the USA's geopolitical and cultural influence globally. Organizations and activists worldwide are concerned that the ruling may inspire anti-abortion legislative and policy attacks in other countries. The ruling has also hindered progressive law reform and the implementation of abortion guidelines in certain African countries. Furthermore, the ruling has created a chilling effect in international policy spaces, empowering anti-abortion actors to under